---
### **Data Bootcamp for Genomic Prediction in Plant Breeding** ###
### **University of Minnesota Plant Breeding Center** ###
#### **June 20 - 22, 2022** ####
---

### **Practical 4: Multitrait Genomic Prediction** ###

<br />
<br />

#### **Source Scripts and Load Data**


In [6]:
WorkDir <- getwd()
setwd(WorkDir)

##Source in functions to be used
source("R_Functions/GS_Pipeline_Jan_2022_FnsApp.R")
source("R_Functions/bootcamp_functions.R")
gc()



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



   *****       ***   vcfR   ***       *****
   This is vcfR 1.12.0 
     browseVignettes('vcfR') # Documentation
     citation('vcfR') # Citation
   *****       *****      *****       *****



Attaching package: 'bWGR'


The following objects are masked from 'package:NAM':

    CNT, GAU, GRM, IMP, KMUP, KMUP2, SPC, SPM, emBA, emBB, emBC, emBL,
    emCV, emDE, emEN, emGWA, emML, emML2, emRR, markov, mkr, mkr2X,
    mrr, mrr2X, mrrFast, wgr



Attaching package: 'emoa'


The following object is masked from 'package:dplyr':

    coalesce



Attaching package: 'MASS'


The following object is masked from 'package:dplyr':

    select



Attaching package: 'sommer'


The following objects are masked from 'package:rrBLUP':

    A.mat, GWAS


Welcome to rTASSEL (version 0.9.26)
 <U+2022> Conside

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,5804502,310.0,8802824,470.2,6426122,343.2
Vcells,10019363,76.5,15504737,118.3,12851603,98.1


#### **Read Genotype File using vcfR** ####

In [7]:

##Load in genotype data. Use package vcfR to read in and work with vcf file.
infileVCF <- "Data/SoyNAM_Geno.vcf"
genotypes_VCF <- read.table(infileVCF)
vcf <- read.vcfR(infileVCF, verbose = FALSE)
vcf

***** Object of Class vcfR *****
5189 samples
20 CHROMs
4,292 variants
Object size: 171.1 Mb
25.41 percent missing data
*****        *****         *****


#### **Convert VCF file format to numerical matrix format.**
#### Final genotype matrix is geno_num

In [6]:
gt <- extract.gt(vcf, element = "GT", as.numeric = F)
fix_T <- as_tibble(getFIX(vcf))
gt2 <- matrix(0, ncol = ncol(gt), nrow = nrow(gt))
colnames(gt2) <- colnames(gt)
rownames(gt2) <- rownames(gt)
gt2a <- apply(gt,2, function(x) gsub("1/1","1",x))
gt2b <- gsub("0[/|]0","0",gt2a)
gt2c <- gsub("[10][/|][10]","0.5",gt2b)
gt2d <- gsub("\\.[/|]\\.","NA",gt2c)

gt2d_num<- apply(gt2d,2,as.numeric)
rownames(gt2d_num)<- rownames(gt2d)
geno_num <- t(gt2d_num)
dim(geno_num)
rm(list=grep("gt2",ls(),value=TRUE))


[1] 5189 4292

#### **Filter Genotypic Data**

In [4]:
##Filter markers on % missing
miss <- function(x){length(which(is.na(x)))}
mrkNA <- (apply(geno_num, MARGIN=2, FUN=miss))/dim(geno_num)[1]
ndx <- which(mrkNA > 0.2)

if (length(ndx)>0) geno_num2 <- geno_num[, -ndx] else geno_num2 <- geno_num

##Filter individuals on % missing
indNA <- (apply(geno_num2, MARGIN=1, FUN=miss))/dim(geno_num2)[2]
ndx2 <- which(indNA > 0.5)

 if(length(ndx2)>0) geno_num3 <- geno_num2[-ndx2, ] else geno_num3 <- geno_num2


##Filter markers based on MAF
maf <- apply(geno_num3, MARGIN=2, FUN=mean, na.rm=T)
ndx3 <- which(maf<0.05 | maf>0.95) 

if (length(ndx3)>0) geno_num4 <- geno_num2[, -ndx3] else geno_num4 <- geno_num3
  
dim(geno_num4)

[1] 5189 1428

#### **Import Phenotypic Data and Merge Geno-Pheno Data**

In [5]:

#pheno <- read.csv("Data/SoyNAM_Pheno.csv")

pheno <- read.csv("Data/SoyNAM_simPheno.csv")

geno_num4_x <- cbind(rownames(geno_num4),geno_num4)
colnames(geno_num4_x)[1]<- "strain"

### Check strain names have same format in pheno and geno 
pheno[,1] <- gsub("[-.]","",pheno[,1])
geno_num4_x[,1] <- gsub("[-.]","",geno_num4_x[,1])

## Merge Geno and Pheno Data
Data <- merge(geno_num4_x,pheno,by="strain",all=TRUE)

## Remove with missing yiled_blup values 

YldNA_Indices <- which(is.na(Data$yield))
if(length(YldNA_Indices) >0){Data_Sub <- Data[-YldNA_Indices,]}else{Data_Sub <- Data}


genoStrain <- unique(as.character(geno_num4_x[,"strain"]))

genoStrainIndices <- which(Data_Sub[,"strain"] %in% genoStrain)
length(genoStrainIndices)
genoIndices <- grep("ss",colnames(geno_num4_x))
initGenoIndx <- genoIndices[1]
finalGenoIndx <- genoIndices[length(genoIndices)]
phenoIndices <- c(1,c((finalGenoIndx+1):ncol(Data_Sub)))

pheno_sub <- Data_Sub[genoStrainIndices,phenoIndices]
geno_num4b <- Data_Sub[genoStrainIndices,c(1,genoIndices)]


uniqueStrainIndices<- which(!duplicated(geno_num4b[,"strain"]))

if(length(uniqueStrainIndices)>0) {geno_num5 <- geno_num4b[uniqueStrainIndices,]}else{geno_num5 <- geno_num4b}

dim(geno_num5)

rm(geno_num4b)
rm(geno_num4)
rm(geno_num3)
rm(geno_num2)

### set 'yield' colname to 'Yield_blup'

yldCol <- which(colnames(pheno_sub) %in% "yield")
colnames(pheno_sub)[yldCol] <- "Yield_blup" 



[1] 55981

[1] 5148 1429

#### **Impute Genotype Table** ###

In [7]:
#### Impute genotable using markov function from 'NAM' package 

geno_imp <- markov(apply(geno_num5[,-1],2,as.numeric))
rownames(geno_imp) <- geno_num5[,"strain"]
dim(geno_imp)

[1] 5148 1428

In [9]:

# Reduce the number of RILs in the dataset simply for the sake of saving time in computation for demonstration (we don't want to spend all of our time watching our computer work!)

ssNdx <- sample.int(n=dim(pheno2)[1], size=5000)
geno_imp_sub <- geno_imp[ssNdx, ]
pheno2_sub <- pheno2[ssNdx, ]






[1] 5145 5145

### **Implement multi-trait genomic prediction models**


### **Use the BGLR package to implement a multi-trait prediction model**

In [ ]:



ETA_MT_BRR <- list(list(X=geno_imp_sub, model='BRR', probIn=.10))
model_MT_BRR <- Multitrait(y=as.matrix(pheno2_sub[, 2:3]), ETA=ETA_MT_BRR, burnIn=1000, nIter=2000, verbose=FALSE)

gebvs_MT_BRR <- model_MT_BRR$ETAHat



# Multi-trait predictions, SpikeSlab in =BGLR

ETA_MT_SS <- list(list(X=geno_imp_sub, model='SpikeSlab', probIn=.10))
model_MT_SS <- Multitrait(y=as.matrix(pheno2_sub[, 2:3]), ETA=ETA_MT_SS, burnIn=1000, nIter=2000, verbose=FALSE)

gebvs_MT_SS <- model_MT_SS$ETAHat


cor(gebvs_MT_BRR[, 1], pheno2_sub$Trait1)
cor(gebvs_MT_BRR[, 2], pheno2_sub$Trait2)


# Single trait predictions

ETA_BB <- list(list(X=geno_imp_sub, model='BRR', probIn=.10))
modelBB <- BGLR(y=pheno2_sub$Trait1, ETA=ETA_BB, burnIn=500, nIter=2000, verbose=FALSE)
bb_gebvs <- modelBB$yHat

cor(bb_gebvs, pheno2_sub$Trait1)



### **Multi-trait prediction using the G-BLUP model execuated in SOMMER**

In [ ]:




A.Tot <- A.mat(geno_imp_sub)

rownames(A.Tot) <- rownames(geno_imp_sub) 
colnames(A.Tot) <- rownames(geno_imp_sub) 

trait <- c("Yield_blup", "Pro_blup")

fm3 <- mmer(as.formula(paste("cbind(",paste(trait,collapse=","),")~1",sep="")),
            random=~vs(RIL,Gu=A.Tot),
            rcov=~units,
            data=pheno2_sub, verbose = TRUE)

fm3 <- mmer(cbind(Yield_blup, Pro_blup)~1,
            random=~vs(RIL,Gu=A.Tot),
            rcov=~units,
            data=pheno2_sub, verbose = TRUE)




#### **Perform 10-fold cross-validation analysis and test predictive ability** 

In [10]:

# This works if total sample size is divisible by 10. If not, need to subset so it is.
ndxShuf <- sample(1:dim(geno_imp_sub)[1], dim(geno_imp_sub)[1])


pheno_shuf <- pheno2_sub[ndxShuf, ]
geno_imp_sub_shuf <- geno_imp_sub[ndxShuf, ]

cnt <- 1:floor(length(ndxShuf)/10)

##Testing multi-trait model with cross-validation
pred_mt_stor <- vector(length=length(ndxShuf))

start_time <- Sys.time()

for (i in 1:10){
  pheno_trn <- pheno_shuf
  pheno_trn[cnt, 2] <- NA
  

  ETA_MT_BRR <- list(list(X=geno_imp_sub_shuf, model='BRR', probIn=.10))
  model_MT_BRR <- Multitrait(y=as.matrix(pheno_trn[, 2:3]), ETA=ETA_MT_BRR, burnIn=1000, nIter=2000, verbose=FALSE)
  
  
  gebvs_MT_BRR <- model_MT_BRR$ETAHat
  
  pred_mt_stor[cnt] <- gebvs_MT_BRR[cnt, 2]
  
  cnt <- cnt + floor(length(ndxShuf)/10)
}

end_time <- Sys.time()
end_time - start_time



# Test single-trait predictions for point of comparison

pred_st_stor <- vector(length=length(ndxShuf))
cnt <- 1:floor(length(ndxShuf)/10)

start_time <- Sys.time()

for (i in 1:10){
  pheno_trn <- pheno_shuf
  pheno_trn[cnt, 2] <- NA
  
  
  ETA_ST_BRR <- list(list(X=geno_imp_sub_shuf, model='BRR', probIn=.10))
  model_ST_BRR <- BGLR(y=pheno_trn[, 2], ETA=ETA_ST_BRR, burnIn=1000, nIter=2000, verbose=FALSE)
  
  
  gebvs_ST_BRR <- model_ST_BRR$yHat
  
  pred_st_stor[cnt] <- gebvs_ST_BRR[cnt]
  
  cnt <- cnt + floor(length(ndxShuf)/10)
}

end_time <- Sys.time()
end_time - start_time


# Estimate correlation between observed yield phenotypes and predicted yield from cross-validation

cor(pred_stor, pheno_shuf[, 3])
plot(pred_stor, pheno_shuf[, 3])



[1] 3 3

[1] 494 494